# Hyperparameters and Model Validation

## Basics steps of supervised machine learning model:
 1. Choose a class of model
2. Choose model hyperparameters
3. Fit model to training data
4. Use model to predict labels for new data

### Model validation

In [12]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut


In [2]:
# The wrong way....


iris = load_iris()

X = iris.data
y = iris.target

X.shape

(150, 4)

In [3]:
# load model

model = KNeighborsClassifier(n_neighbors=1)

# fit the model
model.fit(X, y)

# predict
y_pred = model.predict(X)

In [4]:
# get the accuracy score
accuracy_score(y, y_pred)

1.0

- wrong because it trains and evaluates the same data...

In [10]:
# The right way... train split - Hold out method

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.5)
# model2 = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train, y_train)

y_pred2 = model.predict(X_test)

accuracy_score(y_test, y_pred2)

0.9733333333333334

### Cross validation for model evaluation.

In [11]:
cross_val_score(model, X, y, cv=5)

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

In [16]:
# 
scores = cross_val_score(model, X, y, cv=LeaveOneOut())
print(scores)
print('-----------------------')
print(scores.mean())

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
-----------------------
0.96
